# Creating Household Profiles based on Regulatory Settings

In this notebook, we create optimized household load profiles, based on given regulatory settings. The results are saved in the output folder, where they are later picked up for the grid analysis and the initial analysis of the aggregated profiles.

In [24]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB, quicksum
import datetime
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import statistics
import os

In [25]:
print(gp.gurobi.version())

(9, 5, 0)


In [26]:
# In a first step, the input prices are loaded. We take day-ahead GErman spot market prices
# read EV data with multi-column index
df = pd.read_csv("./input/Hourly_EV_Charging.csv", sep=";", header=[0,1], index_col=0) # only for index, the actual EV data is handled separately
df.index = pd.to_datetime(df.index)

df_price = pd.read_excel("./input/Gro_handelspreise_201901010000_201912312359_Stunde.xlsx", skiprows=9)
df_p = pd.DataFrame()
df_p.index = df.index
# MWh prices are transformed to kWh prices
df_p["Deutschland/Luxemburg [€/kWh]"] = df_price["Deutschland/Luxemburg [€/MWh]"].apply(lambda x: x/1000).values
min_price = df_p["Deutschland/Luxemburg [€/kWh]"].min()
df_p["Deutschland/Luxemburg [€/kWh]"] = df_p["Deutschland/Luxemburg [€/kWh]"].apply(lambda x: x+abs(min_price)) # avoid negative values for optimization
df_p.head()

/Users/leosemmelmann/opt/anaconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Deutschland/Luxemburg [€/kWh]
2019-01-01 00:00:00+00:00,0.11833
2019-01-01 01:00:00+00:00,0.10008
2019-01-01 02:00:00+00:00,0.08593
2019-01-01 03:00:00+00:00,0.08010
2019-01-01 04:00:00+00:00,0.08260


In [27]:
# Now, we also load the 1kW nominal capacity PV generation in Hamelin
df_pv = pd.read_csv("./input/ninja_pv_52.1040_9.3562_uncorrected.csv",skiprows=3)

In [28]:
# Here, we load dataframes with 500 EV, HH and HP profiles. The household config decides which configuration per household is selected
df_ev = pd.read_pickle("./input/2019 Hamelin 500 EV.pkl")
df_hh = pd.read_pickle("./input/2019 Hamelin 500 HH.pkl")
df_hp = pd.read_pickle("./input/2019 Hamelin 500 HP.pkl")
household_config = pd.read_pickle("./input/2019 Hamelin Household Configuration.pkl")

In [29]:
# We use this function to later easily access the outputs of the Gurobi model

def get_results_in_df(m, variableNames, n_timesteps): #  "def name(argumente)" zeigt den Anfang einer Funktion an, die ich später beliebig aufrufen kann
   
    results_df = pd.DataFrame(columns = variableNames, index = [t for t in range(n_timesteps)] )   # Hier wird ein leeres DataFrame erstellt, dass als Spalteneinträge die Namen der Gutobi-variablen hat und als Zeilen die Zeitschritte der Entscheidungsvariablen

    for n in variableNames:                                         #Iteration über alle Zielvariablen
        for t in range(n_timesteps):                                #Iteration über alle Zeitschritte
            VarName = n + f"[{t}]"                                  #Hier wird ein String erstellt, der die Form n[t] hat. Mit diesem wird später die Zielvariable ausgelesen
            try:                                                    #try - except ist eine hilfreiche Methdoe, um Fehler abzufangen. Wenn in der nächsten Zeile ein error passiert, bspw. wegen eines Schreibfehlers in meinen zielvariablen, wird die Funktion nicht abgebrochen sondern weiter ausgeführt.
                results_df.loc[t][n] = m.getVarByName(VarName).x    #Auslesen der Zielvariable
            except:
                pass
            
    return results_df                                               #Hier wird festgelegt, was zurückgegeben wird wenn die Funktion aufgerufen wird. In diesem Fall geben wir das fertige DataFrame mit den ergebnissen der Optimierung zurück

In [30]:
#### Components removed
#handling_fee = 0.02 # https://support.tibber.com/de/articles/4406583-wie-setzt-sich-der-monatliche-tibber-preis-zusammen - removed, previously price component


In [31]:

# we need this function for the rotating grid charge type
def adjust_grid_charges(length, grid_charges, starting_point):
    grid_charges_final = np.full(length, grid_charges/2)  
    offset = starting_point % 2  #starting based on idx inital idx_initial
    for i in range(offset, length, 2):  
        grid_charges_final[i] = grid_charges * 2
    return grid_charges_final



In [33]:
# iterating over various variants

debug = True
household_n = 1 # must be less than 500 

for pricing_type in ["constant","dynamic"]:
    for grid_charge_type in ["volumetric","peak","rotating"]:
        for feed_in_type in ["fit","dynamic"]:
            for grid_charging_allowed in [False]:

                '''
                df_results_hp = pd.DataFrame()
                df_results_hp.index = df_p.index

                df_results_ev = pd.DataFrame()
                df_results_ev.index = df_p.index

                df_results_feedinprofits = pd.DataFrame()
                df_results_feedinprofits.index = df_p.index

                df_results_costs = pd.DataFrame()
                df_results_costs.index = df_p.index
                '''
                # "dynamic" or "constant"
                operation_type = "dynamic"  # "dynamic" or "constant"
                ev_charging_strategy = "early"  # "early" or "spread"; only relevant for operation_type = "constant"
                    # operation_type    |   ev_charging_strategy    |   explanation
                    # constant          |   early                   |   immediate charging when EV is plugged in
                    # constant          |   spread                  |   EV charging is spread over the entire plug-in time

                #grid_charge_type = "volumetric" # "volumetric" or "peak"
                #feed_in_type = "fit" # "fit", "dynamic" or "zero"
                #grid_charging_allowed = True
                
                
                result_path = f"./output/00_pricing_{pricing_type}_operation_{operation_type}_fi_{feed_in_type}_ne_{str(grid_charge_type)}_gridch_{str(grid_charging_allowed)}.pkl"
                
                if debug is False:
                    if os.path.exists(result_path):
                        df_results = pd.read_pickle(result_path)
                    else:
                        df_results = pd.DataFrame()
                        df_results.index = df_p.index
                    
                else:
                    df_results = pd.DataFrame()
                    df_results.index = df_p.index

                for idx_initial, household in household_config.iloc[:household_n].iterrows():
                    if str(idx_initial) not in df_results.columns: # check if already optimized that household
                        hp_load = df_hp[household["heat_pump_profile"]]
                        hh_load = df_hh[household["household_profile"]]
                        ev_load = df_ev[household["household_profile"]].copy().round(5)  # round to avoid numerical issues
                        pv_size = household["pv_power"]
                        bess_size = household["bess_capacity"]
                        max_bess_power = household["bess_power"]

                        # Initialize your environment and model
                        env = gp.Env(empty=True)
                        env.setParam("OutputFlag", 1)
                        env.start()
                        model = gp.Model("test", env=env)
                        model.setParam('MIPGap', 0.01) 
                        model.setParam('TimeLimit', 300)

                        length = len(df_p)
                        days = int(length/24)

                        ev_load["kWh"] = ev_load["Wh"].apply(lambda x:x/1000).values  # W to kW
                        # real_ev_charging[-24:] = 0 # setting the last 24 values of ev charging = 0 to avoid model infeasibility; this should no longer be necessary with the new EV data as only full charging sessions in 2019 (UTC time) are considered (see Preprocessing_EV_data > "only consider charging sessions in defined time window")
                        max_ev_charging = (ev_load["kWh"] / ev_load["share_of_hour"]).max()  # maximum ev charge is set at the maximum value


                        pv_size = bess_size
                        pv_load = df_pv["electricity"].apply(lambda x: x*pv_size).values
                        min_bess_energy = 0.05*bess_size
                        feed_in_tariff_fixed = 0.1187 # 0.07
                        grid_charge = 0.0722 # https://www.bundesnetzagentur.de/SharedDocs/Mediathek/Monitoringberichte/Monitoringbericht_VerbraucherKennzahlen2019.pdf
                        peak_power_charge = 67.94 #https://www.avacon-netz.de/content/dam/revu-global/avacon-netz/documents/netzentgelte-strom
                        bess_efficiency = 0.95
                        guarantee_cycles = 365
                        max_amount_blocked_hours = 6

                        # DETERMINING PRICES BASED ON TYPE
                        if pricing_type == "dynamic":
                            prices = df_p["Deutschland/Luxemburg [€/kWh]"].apply(lambda x: x).values
                        elif pricing_type == "constant":
                            average_price = df_p["Deutschland/Luxemburg [€/kWh]"].mean()
                            prices = np.full(length, average_price)
                        else:
                            raise ValueError("Pricing type not defined.")
                            
                        # DETERMINING GRID CHARGES TYPE
                        if grid_charge_type == "volumetric":
                            grid_charges  = np.full(length, grid_charge)
                        
                        elif grid_charge_type == "rotating":
                            found_starting_point = False
                            starting_point=0
                            while found_starting_point is False:
                                modulo_value = (idx_initial+starting_point)%2
                                if modulo_value != 0:
                                    starting_point = starting_point+1
                                else:
                                    found_starting_point = True
                            print(idx_initial)
                            print(f"Starting point is: {starting_point}")
                            grid_charges_final = adjust_grid_charges(length, grid_charge, starting_point)
                            grid_charges = grid_charges_final
                        elif grid_charge_type =="peak":
                            grid_charges = np.full(length, 0)
                        else:
                            raise ValueError("Grid charge type not properly defined.")
                                
                                

                        # DETERMINING FEED-IN REMUNERATION

                        if feed_in_type == "fit":
                            feed_in_tariff = np.full(length, feed_in_tariff_fixed)
                        elif feed_in_type == "dynamic":
                            feed_in_tariff = df_p["Deutschland/Luxemburg [€/kWh]"].values
                        elif feed_in_type == "zero":
                            feed_in_tariff = np.full(length, 0)
                        else:
                            raise ValueError("Feed-in remuneration not defined.")


                        # transform hh and hp load to kW
                        real_hh_load = hh_load.apply(lambda x:x/1000).values
                        real_hp_load = hp_load.apply(lambda x:x/1000).values
                        max_hp_load = real_hp_load.max()

                        # Initialize variables
                        #if operation_type == "dynamic":
                        opt_ev_charging = model.addVars([t for t in range(length)], lb=0,vtype=GRB.CONTINUOUS, name="p_ch_opt")
                        opt_hp_load = model.addVars([t for t in range(length)], lb=0, vtype=GRB.CONTINUOUS, name="p_hp_opt")
                        opt_bess_charging = model.addVars([t for t in range(length)], lb=0,vtype=GRB.CONTINUOUS, name="p_bess_ch_opt")
                        opt_bess_discharging = model.addVars([t for t in range(length)], lb=0,vtype=GRB.CONTINUOUS, name="p_bess_dch_opt")
                        opt_net_energy = model.addVars([t for t in range(length)],lb=0,vtype=GRB.CONTINUOUS,name="opt_net_energy")
                        buy_energy = model.addVars(length, vtype=GRB.BINARY, name="buy_energy") # positive when grid intake, zero when feed-out
                        e_bess = model.addVars([t for t in range(length)], vtype=GRB.CONTINUOUS, name="e_bess")


                        energy_costs = model.addVars([t for t in range(length)], vtype=GRB.CONTINUOUS,name = "energy_costs")
                        feedin_profits = model.addVars([t for t in range(length)], vtype=GRB.CONTINUOUS,name = "feedin_profits")

                        opt_feed_out_pv = model.addVars([t for t in range(length)], vtype=GRB.CONTINUOUS, name="opt_feed_out_pv")
                        opt_internaluse_pv = model.addVars([t for t in range(length)], vtype=GRB.CONTINUOUS, name="opt_internaluse_pv")


                        block_hp = model.addVars([t for t in range(length)], vtype=gp.GRB.BINARY, name = "blocked_heatpump_event")
                        block_hp_hour = model.addVars([t for t in range(length)], vtype=gp.GRB.BINARY, name = "blocked_heatpump_hour")

                        # regulatory options
                        max_net_energy = model.addVar(name="max_net_energy")


                        #### ELECTRIC VEHICLE
                        # ensure that charging requirements are met

                        if operation_type == "dynamic":
                            for ts_start in ev_load[ev_load["start"] > 0].index:
                                hours_until_end = int(ev_load.loc[ts_start, "hours_until_end"])
                                # ts_start is the timestamp when the charging session starts ; convert to integer index
                                idx_start = ev_load.index.get_loc(ts_start)
                                idx_end = idx_start + hours_until_end
                                # ensure that the energy demand of each charging session is met
                                model.addConstr(quicksum(opt_ev_charging[t] for t in range(idx_start, idx_end + 1)) ==
                                                quicksum(ev_load.iloc[t]["kWh"] for t in range(idx_start, idx_end + 1)), "energy_charging_session")

                            # ensure maximum empirical EV charging is never exceeded
                            # if the EV is only plugged in for a fraction of the hour (i.e., share_of_hour < 1),
                            # it can also only charge for that fraction
                            model.addConstrs(opt_ev_charging[t] <= max_ev_charging * ev_load.iloc[t]["share_of_hour"] for t in range(length))
                        elif operation_type == "constant" and ev_charging_strategy == "early":
                            # run helper optimization to define charging power in each time step
                            model_h = gp.Model("ev_early_charging", env=env)
                            opt_ev_charging_h = model_h.addVars([t for t in range(length)], vtype=GRB.CONTINUOUS, name="p_ch_opt_h")
                            coeff_early_charging = [0] * len(ev_load)  # used in the objective function
                            for ts_start in ev_load[ev_load["start"] > 0].index:
                                hours_until_end = int(ev_load.loc[ts_start, "hours_until_end"])
                                # ts_start is the timestamp when the charging session starts ; convert to integer index
                                idx_start = ev_load.index.get_loc(ts_start)
                                idx_end = idx_start + hours_until_end
                                # ensure that the energy demand of each charging session is met
                                model_h.addConstr(quicksum(opt_ev_charging_h[t] for t in range(idx_start, idx_end + 1)) ==
                                                quicksum(ev_load.iloc[t]["kWh"] for t in range(idx_start, idx_end + 1)), "energy_charging_session")
                                coeff_early_charging[idx_start:idx_end + 1] = range(hours_until_end + 1)

                            # ensure maximum empirical EV charging is never exceeded
                            model_h.addConstrs(opt_ev_charging_h[t] <= max_ev_charging * ev_load.iloc[t]["share_of_hour"] for t in range(length))

                            # objective function
                            model_h.setObjective(quicksum(opt_ev_charging_h[t] * coeff_early_charging[t] for t in range(length)), GRB.MINIMIZE)
                            model_h.optimize()

                            # assign results to the main model
                            model.addConstrs(opt_ev_charging[t] == opt_ev_charging_h[t].x for t in range(length))
                        else:
                            # EV load is spread over the entire plug-in time; this is how input data is formatted
                            model.addConstrs(opt_ev_charging[t] == ev_load.iloc[t]["kWh"] for t in range(length))

                        #### HEAT PUMP
                        # ensure maximum three blocking windows and unblocking for two hours after blocking

                        if operation_type == "dynamic":

                            for idx in range(days): 
                                idx_start = idx * 24
                                idx_end = (idx + 1) * 24

                                model.addConstr(quicksum(block_hp_hour[t] for t in range(idx_start,idx_end)) <= max_amount_blocked_hours, "blocked_hours_per_day_{}".format(idx)) # LEO VORSCHLAG: LIMIT BLOCKING HOURS PER DAY
                                #model.addConstr(quicksum(block_hp[t] for t in range(idx_start,idx_end)) <= 3, "blocked_hours_per_day_{}".format(idx)) # KATHI VORSCHLAG 

                            #for t in range(length-1): # KATHI VORSCHLAG
                                #model.addConstr(block_hp_hour[t] + block_hp_hour[t+1] <= 2 * (1-block_hp[t])) # KATHI VORSCHLAG
                            
                            for t in range(length-4): # LEO VORSCHLAG
                                model.addConstr(block_hp_hour[t] + block_hp_hour[t+1] + block_hp_hour[t+2] + block_hp_hour[t+3]<= 2 ) # LEO VORSCHLAG
                            
                            
                            #for t in range(1,length-1): # KATHI VORSCHLAG
                            #    model.addConstr((block_hp[t]>=block_hp_hour[t-1]+block_hp_hour[t])) # has to be 1 if there is a switch from blocked to unblocked; limit the sum of s_t over one day to 3 --> maximum three blocking events; KATHI VORSCHLAG

                            for t in range(length-1):
                                model.addConstr(opt_hp_load[t]<=max_hp_load)
                                model.addConstr((opt_hp_load[t]>=(1-block_hp_hour[t])*real_hp_load[t]))            
                                model.addConstr(opt_hp_load[t] <= (1 - block_hp_hour[t]) * max_hp_load) # making sure that albeit the flexible constraint blocking hours lead to complete blocking

                            # ensure that the sum of heat pump load every 6 hours remains in the same range
                            for idx in range(days * 4):
                                idx_start = idx * 6
                                idx_end = (idx + 1) * 6
                                if idx_end > length:
                                    idx_end = length  # Ensure we don't exceed the bounds
                                model.addConstr(quicksum(opt_hp_load[t] for t in range(idx_start, idx_end)) == quicksum(real_hp_load[t] for t in range(idx_start, idx_end)), "hp_load_6hr_block_{}".format(idx))

                        elif operation_type == "constant":
                            model.addConstrs(opt_hp_load[t] == real_hp_load[t] for t in range(length))

                        #### BATTERY STORAGE
                        model.addConstr(e_bess[0]==min_bess_energy)

                        for t in range(length):
                            model.addConstr(opt_bess_charging[t] * opt_bess_discharging[t] == 0, "mutual_exclusivity_" + str(t))
                            model.addConstr(opt_net_energy[t] * opt_feed_out_pv[t] == 0, "mutual_exclusivity_pv_" + str(t)) # mutual exclusivity for pv feed out and net energy 
                            model.addConstr(opt_bess_charging[t]<=max_bess_power)
                            model.addConstr(opt_bess_discharging[t]<=max_bess_power)
                            if grid_charging_allowed is not True:
                                model.addConstr(opt_bess_charging[t]<=opt_internaluse_pv[t])


                        for t in range(1, length):
                            model.addConstr(e_bess[t]==e_bess[t-1]+opt_bess_charging[t-1]*bess_efficiency-opt_bess_discharging[t-1]/bess_efficiency)
                            model.addConstr(e_bess[t]>=min_bess_energy)
                            model.addConstr(e_bess[t]<=bess_size)

                        model.addConstr(quicksum(opt_bess_discharging[t] + opt_bess_charging[t] for t in range(length)) <= bess_size*2*guarantee_cycles, "guarantee_cycles")


                        ### OPTI
                        # calculate net energy of household
                        if operation_type == "dynamic":
                            model.addConstrs(opt_net_energy[t] == (real_hh_load[t]+opt_ev_charging[t]+opt_bess_charging[t]*bess_efficiency-opt_bess_discharging[t]/bess_efficiency+opt_hp_load[t]-opt_internaluse_pv[t])for t in range(length))
                        elif operation_type == "constant" and ev_charging_strategy == "early":
                            model.addConstrs(opt_net_energy[t] == (real_hh_load[t]+opt_ev_charging[t]+opt_bess_charging[t]-opt_bess_discharging[t]+real_hp_load[t]-opt_internaluse_pv[t])for t in range(length))
                        else:
                            model.addConstrs(opt_net_energy[t] == (real_hh_load[t]+ev_load.iloc[t]["kWh"]+opt_bess_charging[t]-opt_bess_discharging[t]+real_hp_load[t]-opt_internaluse_pv[t]) for t in range(length))


                        # regulatory cases
                        for t in range(length):
                            if grid_charge_type == "peak":
                                model.addConstr(max_net_energy >= opt_net_energy[t], name=f"max_constraint_{t}")

                            model.addConstr(opt_net_energy[t] >= 0)


                        # energy costs
                        for t in range(length):
                            model.addConstr((opt_internaluse_pv[t]+opt_feed_out_pv[t])==pv_load[t]) # TODO: adding efficiency
                            model.addConstr(energy_costs[t] == ((opt_net_energy[t]) * (prices[t]+grid_charges[t]) ), "calc_energy_costs_" + str(t))
                            model.addConstr(feedin_profits[t] == opt_feed_out_pv[t]*feed_in_tariff[t], "calc_energy_profits_" + str(t))

                        # objective function
                        if grid_charge_type == "peak":
                            model.setObjective(quicksum(energy_costs[t]-feedin_profits[t] for t in range(length))+max_net_energy*peak_power_charge, GRB.MINIMIZE) 
                        else:
                            model.setObjective(quicksum(energy_costs[t]-feedin_profits[t] for t in range(length)), GRB.MINIMIZE) 

                        # optimize
                        model.optimize()

                        # get results
                        vn = ["opt_net_energy","energy_costs","p_ch_opt","p_hp_opt","blocked_heatpump_event","blocked_heatpump_hour","p_bess_ch_opt","p_bess_dch_opt","opt_feed_out_pv","opt_internaluse_pv","e_bess","feedin_profits"] #Hier müssen die Namen meiner Gurobi-Zielvariablen stehen (siehe Optimierung)
                        n = length
                        temp_results = get_results_in_df(model, vn, n)
                        df_results[str(idx_initial)] = temp_results["opt_net_energy"].values - temp_results["opt_feed_out_pv"].values
                        #df_results_hp[str(idx_initial)] = temp_results["p_hp_opt"].values
                        #df_results_ev[str(idx_initial)] = temp_results["p_ch_opt"].values

                        if df_results.isna().any().any():
                            raise ValueError(f"Results for {grid_charge_type} {feed_in_type} {str(grid_charging_allowed)} contain NaN values, indicating infeasibility of some models.")
                        else:
                            if debug is False:
                                df_results.to_pickle(result_path)
                            #df_results_hp.to_pickle(f"./output/detailed_consumption/HP_00_pricing_{pricing_type}_operation_{operation_type}_fi_{feed_in_type}_ne_{str(grid_charge_type)}_gridch_{str(grid_charging_allowed)}.pkl")
                            #df_results_ev.to_pickle(f"./output/detailed_consumption/EV_00_pricing_{pricing_type}_operation_{operation_type}_fi_{feed_in_type}_ne_{str(grid_charge_type)}_gridch_{str(grid_charging_allowed)}.pkl")
                            #df_results_feedinprofits.to_pickle(f"./output/detailed_financials/FI_00_pricing_{pricing_type}_operation_{operation_type}_fi_{feed_in_type}_ne_{str(grid_charge_type)}_gridch_{str(grid_charging_allowed)}.pkl")
                            #df_results_costs.to_pickle(f"./output/detailed_financials/Costs_00_pricing_{pricing_type}_operation_{operation_type}_fi_{feed_in_type}_ne_{str(grid_charge_type)}_gridch_{str(grid_charging_allowed)}.pkl")
                    else:
                        print(str(idx_initial)+" has been already optimized for that case.")


Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-20
Set parameter MIPGap to value 0.01
Set parameter TimeLimit to value 300
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 142167 rows, 113881 columns and 327679 nonzeros
Model fingerprint: 0xf3bfb03a
Model has 17520 quadratic constraints
Variable types: 87601 continuous, 26280 integer (26280 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+01]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-02, 7e+03]
Presolve removed 80435 rows and 59337 columns
Presolve time: 0.55s
Presolved: 61732 rows, 54544 columns, 201727 nonzeros
Variable types: 37684 continuous, 16860 integer (16860 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.08

ValueError: Results for volumetric dynamic False contain NaN values, indicating infeasibility of some models.

In [19]:
feed_in_type

'fit'

In [ ]:
'''if df_results.isna().any().any():
    print(f"{}")
    raise ValueError(f"Results for {} contain NaN values, indicating infeasibility of some models.")
else:
    df_results.to_pickle(f"./output/00_pricing_{pricing_type}_operation_{operation_type}_fi_{feed_in_type}_ne_{str(grid_charge_type)}_gridch_{str(grid_charging_allowed)}.pkl")
    df_results_hp.to_pickle(f"./output/detailed_consumption/HP_00_pricing_{pricing_type}_operation_{operation_type}_fi_{feed_in_type}_ne_{str(grid_charge_type)}_gridch_{str(grid_charging_allowed)}.pkl")
    df_results_ev.to_pickle(f"./output/detailed_consumption/EV_00_pricing_{pricing_type}_operation_{operation_type}_fi_{feed_in_type}_ne_{str(grid_charge_type)}_gridch_{str(grid_charging_allowed)}.pkl")

    '''

In [ ]:
temp_results["opt_net_energy"].max()

In [ ]:
grid_charges_final[:25]

In [ ]:
df_p.iloc[:25]

In [ ]:
prices[:25]



In [ ]:
temp_results.index = pd.to_datetime(df_p.index)
temp_results.index = temp_results.index.tz_localize(None)
temp_results.index = temp_results.index.strftime('%Y-%m-%d %H:%M:%S')

temp_results

In [ ]:

temp_results["p_ch_real"] = ev_load.iloc[t]["kWh"]
temp_results["Deutschland/Luxemburg [€/kWh]"] = prices
temp_results["p_pv"] = pv_load
temp_results["p_hp_real"] = real_hp_load
temp_results["p_hh_real"] = real_hh_load
temp_results["real_energy"] = temp_results["p_hh_real"] + temp_results["p_hp_real"] + temp_results["p_ch_real"]
temp_results["opt_energy"] = temp_results["p_hh_real"] + temp_results["p_hp_opt"] + temp_results["p_ch_opt"]

In [ ]:
# Selecting day for analysis
day = 2

In [ ]:
plt.rcParams.update({'font.size': 18})
plt.rcParams["figure.figsize"] = (12,6)


In [ ]:
fig, ax1 = plt.subplots()



color1 = 'tab:red'
color2 = 'tab:green'
# First axis for p_ch_real and p_ch_opt
ax1.set_xlabel('Hour of the day')
ax1.set_ylabel('Energy consumption [kWh]', color=color2)
#ax1.plot(temp_results.index[day*24:(day+1)*24], temp_results['p_ch_real'].iloc[day*24:(day+1)*24], color=color1, label='p_ch_real')
ax1.plot(temp_results.index[day*24:(day+1)*24], temp_results['p_ch_opt'].iloc[day*24:(day+1)*24], color=color2, label='p_ch_opt')
ax1.tick_params(axis='y', labelcolor=color2)
ax1.legend(loc='upper left')

ax2 = ax1.twinx()  
color3 = 'tab:blue'
ax2.set_ylabel('Deutschland/Luxemburg [€/kWh]', color=color3)
ax2.plot(temp_results.index[day*24:(day+1)*24], temp_results['Deutschland/Luxemburg [€/kWh]'].iloc[day*24:(day+1)*24], color=color3)
ax2.tick_params(axis='y', labelcolor=color3)

ax1.set_xticklabels(ax1.get_xticks(), rotation=90)

fig.tight_layout()  # for a neat layout
plt.show()



In [ ]:
temp_results.index[day*24:(day+1)*24]

In [ ]:
fig, ax1 = plt.subplots()



color1 = 'tab:red'
color2 = 'tab:green'
# First axis for p_ch_real and p_ch_opt
ax1.set_xlabel('Hour of the day')
ax1.set_ylabel('Energy consumption [kWh]', color=color2)
ax1.plot(temp_results.index[day*24:(day+1)*24], temp_results['p_hp_real'].iloc[day*24:(day+1)*24], color=color1, label='p_hp_real')
ax1.plot(temp_results.index[day*24:(day+1)*24], temp_results['p_hp_opt'].iloc[day*24:(day+1)*24], color=color2, label='p_hp_opt')
ax1.tick_params(axis='y', labelcolor=color2)
ax1.legend(loc='upper left')

ax2 = ax1.twinx()  
color3 = 'tab:blue'
ax2.set_ylabel('Deutschland/Luxemburg [€/kWh]', color=color3)
ax2.plot(temp_results.index[day*24:(day+1)*24], temp_results['Deutschland/Luxemburg [€/kWh]'].iloc[day*24:(day+1)*24], color=color3)
ax2.tick_params(axis='y', labelcolor=color3)
ax1.set_xticklabels(ax1.get_xticks(), rotation=90)


fig.tight_layout()  # for a neat layout
plt.show()



In [ ]:
fig, ax1 = plt.subplots()



color1 = 'tab:red'
color2 = 'tab:green'
# First axis for p_ch_real and p_ch_opt
ax1.set_xlabel('Hour of the day')
ax1.set_ylabel('Energy [kWh]', color=color1)
ax1.plot(temp_results.index[day*24:(day+1)*24], temp_results['p_pv'].iloc[day*24:(day+1)*24], color=color1, label='p_pv_real')
ax1.plot(temp_results.index[day*24:(day+1)*24], temp_results['opt_feed_out_pv'].iloc[day*24:(day+1)*24], color=color2, label='opt_feed_out_pv')
ax1.tick_params(axis='y', labelcolor=color1)
ax1.legend(loc='upper left')

ax2 = ax1.twinx()  
color3 = 'tab:blue'
ax2.set_ylabel('Deutschland/Luxemburg [€/kWh]', color=color3)
ax2.plot(temp_results.index[day*24:(day+1)*24], temp_results['Deutschland/Luxemburg [€/kWh]'].iloc[day*24:(day+1)*24], color=color3)
ax2.tick_params(axis='y', labelcolor=color3)

ax1.set_xticklabels(ax1.get_xticks(), rotation=90)

fig.tight_layout()  # for a neat layout
plt.show()



In [ ]:
fig, ax1 = plt.subplots()



color1 = 'tab:red'
color2 = 'tab:green'
# First axis for p_ch_real and p_ch_opt
ax1.set_xlabel('Hour of the day')
ax1.set_ylabel('Energy [kWh]', color=color1)
ax1.plot(temp_results.index[day*24:(day+1)*24], temp_results['opt_internaluse_pv'].iloc[day*24:(day+1)*24], color=color1, label='opt_internaluse_pv')
ax1.plot(temp_results.index[day*24:(day+1)*24], temp_results['opt_feed_out_pv'].iloc[day*24:(day+1)*24], color=color2, label='opt_feed_out_pv')
ax1.tick_params(axis='y', labelcolor=color1)
ax1.legend(loc='upper left')

ax2 = ax1.twinx()  
color3 = 'tab:blue'
ax2.set_ylabel('Deutschland/Luxemburg [€/kWh]', color=color3)
ax2.plot(temp_results.index[day*24:(day+1)*24], temp_results['Deutschland/Luxemburg [€/kWh]'].iloc[day*24:(day+1)*24], color=color3)
ax2.tick_params(axis='y', labelcolor=color3)
ax1.set_xticklabels(ax1.get_xticks(), rotation=90)

fig.tight_layout()  #  for a neat layout
plt.show()



In [ ]:
fig, ax1 = plt.subplots()



color1 = 'tab:red'
color2 = 'tab:green'
# First axis for p_ch_real and p_ch_opt
ax1.set_xlabel('Hour of the day')
ax1.set_ylabel('Energy [kWh]', color=color1)
ax1.plot(temp_results.index[day*24:(day+1)*24], temp_results['p_bess_ch_opt'].iloc[day*24:(day+1)*24], color=color1, label='p_bess_ch_opt')
ax1.plot(temp_results.index[day*24:(day+1)*24], temp_results['p_bess_dch_opt'].iloc[day*24:(day+1)*24], color=color2, label='p_bess_dch_opt')
ax1.tick_params(axis='y', labelcolor=color1)
ax1.legend(loc='upper left')

ax2 = ax1.twinx()  
color3 = 'tab:blue'
ax2.set_ylabel('Deutschland/Luxemburg [€/kWh]', color=color3)
ax2.plot(temp_results.index[day*24:(day+1)*24], temp_results['Deutschland/Luxemburg [€/kWh]'].iloc[day*24:(day+1)*24], color=color3)
ax2.tick_params(axis='y', labelcolor=color3)
ax1.set_xticklabels(ax1.get_xticks(), rotation=90)

fig.tight_layout()  # for a neat layout
plt.show()





In [ ]:
fig, ax1 = plt.subplots()



color1 = 'tab:red'
color2 = 'tab:green'
# First axis for p_ch_real and p_ch_opt
ax1.set_xlabel('Hour of the day')
ax1.set_ylabel('Energy consumption [kWh]', color=color1)
ax1.plot(temp_results.index[day*24:(day+1)*24], temp_results['real_energy'].iloc[day*24:(day+1)*24], color=color1, label='real_energy')
ax1.plot(temp_results.index[day*24:(day+1)*24], temp_results['opt_energy'].iloc[day*24:(day+1)*24], color=color2, label='opt_energy')
ax1.tick_params(axis='y', labelcolor=color1)
ax1.legend(loc='upper left')

ax2 = ax1.twinx()  
color3 = 'tab:blue'
ax2.set_ylabel('Deutschland/Luxemburg [€/kWh]', color=color3)
ax2.plot(temp_results.index[day*24:(day+1)*24], temp_results['Deutschland/Luxemburg [€/kWh]'].iloc[day*24:(day+1)*24], color=color3)
ax2.tick_params(axis='y', labelcolor=color3)

fig.tight_layout()  # for a neat layout
ax1.set_xticklabels(ax1.get_xticks(), rotation=90)

plt.show()





In [ ]:
fig, ax1 = plt.subplots()



color1 = 'tab:red'
color2 = 'tab:green'
# First axis for p_ch_real and p_ch_opt
ax1.set_xlabel('Time')
ax1.set_ylabel('p_ch_real & p_ch_opt', color=color1)
ax1.plot(temp_results.index[day*24:(day+1)*24], temp_results['opt_net_energy'].iloc[day*24:(day+1)*24], color=color1, label='opt_net_energy')
#ax1.plot(temp_results.index[day*24:(day+1)*24], temp_results['opt_energy'].iloc[day*24:(day+1)*24], color=color2, label='opt_energy')
ax1.tick_params(axis='y', labelcolor=color1)
ax1.legend(loc='upper left')

ax2 = ax1.twinx()  
color3 = 'tab:blue'
ax2.set_ylabel('Deutschland/Luxemburg [€/kWh]', color=color3)
ax2.plot(temp_results.index[day*24:(day+1)*24], temp_results['Deutschland/Luxemburg [€/kWh]'].iloc[day*24:(day+1)*24], color=color3)
ax2.tick_params(axis='y', labelcolor=color3)
ax1.set_xticklabels(ax1.get_xticks(), rotation=90)

fig.tight_layout()  # for a neat layout
plt.show()





In [16]:
# Debugging

model.computeIIS()
model.write("model.ilp")


IIS runtime: 1.92 seconds (1.15 work units)


GurobiError: Cannot compute IIS on a feasible model

In [ ]:
household_config.iloc[:50]

In [ ]:
# Debugging 

if df_results.isna().any().any():
    columns_with_na = df_results.isna().any()
    columns_with_na_names = columns_with_na[columns_with_na].index.tolist()
    print(columns_with_na_names)
    
temp_ev_df = pd.DataFrame(ev_load)
temp_ev_df.index = pd.to_datetime(temp_ev_df.index)
temp_ev_df.resample("D").sum()

failed_hhs = []
transposed_config = household_config.T
for hh in columns_with_na_names:
    failed_hhs.append(transposed_config[int(hh)])
    
failed_hhs_df = pd.DataFrame(failed_hhs)


#household_config[household_config["heat_pump_profile"]=="148_SFH34_P_TOT"].index
for i in household_config.index:
    if household_config.index[i]==148:
        print(i)


failed_hhs_df

In [ ]:
df_ev_comparison = pd.DataFrame({"dynamic": df_results_ev_dynamic["p_ch_opt"].values,
                                 "constant_early": df_results_ev_constant_early["p_ch_opt"].values,
                                 "constant_spread": df_results_ev_constant_spread["p_ch_opt"].values,
                                 "input": ev_load["kWh"].values,
                                 "prices": prices},
                                index=df_ev.index)

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 4))
color1 = 'tab:red'
color2 = 'tab:green'
color3 = 'tab:orange'
color4 = 'tab:blue'

n_days = 14
start_day = 1
df_ = df_ev_comparison.iloc[start_day*24:start_day*24 + n_days*24]
df_[["dynamic", "constant_early", "constant_spread"]].plot(ax=ax1, color=[color1, color2, color3])
ax1.set_ylabel('Charging power [kW]')
ax1.legend(loc='upper left')

ax2 = ax1.twinx()
ax2.set_ylabel('Deutschland/Luxemburg [€/kWh]', color=color4)
df_[["prices"]].plot(ax=ax2, color=[color4], alpha=0.5)
ax2.tick_params(axis='y', labelcolor=color4)
ax2.legend().remove()

fig.tight_layout()

In [17]:
df_results

,158,262,458,348,356,136,211,236,166,333,...,111,217,164,70,407,289,249,338,470,318
2019-01-01 00:00:00+00:00,1.248182,3.873068,0.201543,2.395007,2.477965,4.042532,1.564321,3.906959,4.140716,11.606914,...,0.8577,1.125855,0.165621,0.062566,0.442608,1.967809,0.169156,0.557912,0.104099,NaN
2019-01-01 01:00:00+00:00,4.654666,3.756669,9.903357,9.473352,3.002625,7.188406,3.636788,8.206975,11.291425,0.177802,...,0.45522,0.159344,1.387093,2.357121,0.396434,0.100524,0.268335,1.170177,2.205573,NaN
2019-01-01 02:00:00+00:00,0.139974,4.194957,4.741807,0.322103,2.212881,0.19406,1.887628,0.182594,3.189367,5.309733,...,1.43131,0.857055,0.206584,0.077082,1.094107,0.400867,0.307633,0.111544,0.116627,NaN
2019-01-01 03:00:00+00:00,1.538806,4.805189,5.260218,3.469259,3.197579,4.255108,2.919821,4.49515,2.847223,0.17688,...,0.503318,0.191914,0.285917,0.651311,0.383412,0.099765,0.740709,0.45561,1.166539,NaN
2019-01-01 04:00:00+00:00,1.88284,3.45823,4.788622,0.240473,2.294643,3.783663,3.823444,3.765532,0.20326,3.544323,...,0.640314,0.460027,0.681939,0.092207,0.595363,0.575555,0.870569,0.622429,0.090472,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 19:00:00+00:00,4.142481,1.793684,5.908466,5.721223,4.874935,3.207544,0.143701,2.431253,2.863074,0.0,...,0.0,0.0,0.486758,3.070075,0.0,0.0,0.0,2.360294,4.020522,NaN
2019-12-31 20:00:00+00:00,-0.0,0.450683,-0.0,4.406545,0.0,0.0,4.135914,-0.0,1.814581,3.226909,...,0.157924,0.499635,0.0,0.186163,1.46176,0.646612,0.195631,0.0,0.0,NaN
2019-12-31 21:00:00+00:00,4.210629,2.741655,4.518551,5.263739,6.420833,3.519057,0.472259,3.428005,2.921799,0.0,...,0.0,0.0,0.166535,2.711186,-0.0,0.0,0.0,2.371254,2.475374,NaN
2019-12-31 22:00:00+00:00,0.840446,0.0,0.0,1.277676,0.0,0.0,2.466583,0.0,0.224754,3.742975,...,0.133789,0.0,0.0,1.076749,0.515653,0.153092,0.143112,2.456682,0.0,NaN
